article : https://arxiv.org/pdf/1307.0048.pdf

L'idée du projet est d'implementer un algorithme mapreduce sur la régression linéaire pénalisée, quand $X \in \mathbb{R}^{n \times p}$ avec $p << n$.

Cela correspond à un type de problème ou le nombre de features $p$ (les caractéristiques d'un individu ou un produit) est assez petit et il est envisageable de les stocker en mémoire, alors que la taille du dataset $n$ et très grande et on voudrait faire du calcul distributé dessus

L'idée de l'agorithme est alors d'exprimer la quantité à minimiser en fonction des matrices ou des vecteurs dont la dimension est une fonction de $p$ ($p\times p$ ou $p$ en fait). Et calculer ces quantitées à partir de $X \in \mathbb{R}^{n \times p}$ en faisant une reduction sur $n$ (qui est la taille de notre dataset)

In [1]:
import numpy as np
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz

p = 10
n = 100
cov = toeplitz(0.5 ** np.arange(p))
A = multivariate_normal(np.zeros(p), cov, 100)
A_sc = sc.parallelize(A)

# Algorithm 1

En notant $X_c \in \mathbb{R}^{n \times p }$ la matrice centrée réduite de $X$. On a que minimiser:
$$||Y - \alpha \mathbb{1} - X \beta||_2 + p_\lambda(\beta)$$

Revient au même que minimiser:
$$\begin{align}
||Y - \hat{\alpha} \mathbb{1} - X_c \hat{\beta}||_2 + p_\lambda(\hat{\beta})
\end{align}$$
Avec le changement de variable:

$$\begin{align} 
\hat{\alpha}&= \alpha + \left(\bar{X_1}, \dots, \bar{X_p}\right) \beta \\
\hat{\beta} &= D \beta
\end{align}
 $$
 
 avec D la matrice diagonale des déviations standards.
 
 
 Comme maintenant les variables sont centrées, la minimisation en $\hat{\alpha}$ donne $\hat{\alpha} = \bar{Y}$, et:
 $$\begin{align}
 \hat{\beta}^* &= \arg\min_{\hat{\beta}} ||Y - \hat{\alpha} \mathbb{1} - X_c \hat{\beta}||_2 + p_\lambda(\hat{\beta}) \\
             &= \arg\min_{\hat{\beta}} ||(Y - \hat{\alpha} \mathbb{1})||_2^2  + ||X_c \hat{\beta}||_2^2 - 2(Y - \hat{\alpha} \mathbb{1})^T X_c \hat{\beta}  + p_\lambda(\hat{\beta}) \\
             &= \arg\min_{\hat{\beta}} ||X_c \hat{\beta}||_2^2 - 2(Y - \hat{\alpha} \mathbb{1})^T X_c \hat{\beta}  + p_\lambda(\hat{\beta}) \\
             &= \arg\min_{\hat{\beta}} \hat{\beta}^TX_c^T X_c \hat{\beta} - 2Y^T X_c\hat{\beta}  + p_\lambda(\hat{\beta})
 \end{align}$$
 
 $X_c^TX_c, Y^TX_c$ sont une matrice de taille $p \times p$ et un vecteur de taille $p$. On peut donc par hypothèse les stocker en mémoire et résoudre ce problème par une des méthodes d'optimisation classiques (coordinate descent par exemple).
 
 Les quantités qu'on doit calculer sont $X_c^TX_c$ qui est la matrice de correlation de $X$. $Y^TX_c$ et $(\bar{X_1}, \dots, \bar{X_p})$ (pour faire le changement de variables inverse).
 
 # En fait pas sûr de mon truc car quand on fait du cross validation centrée sur k-1 partition  с'est pas pareil que de centrer sur tout


In [2]:
def PenalizedLR_MR(Data, k, lambdas):
    """
    Data: an RDD each rows of which is a tuple (x, y)
    k: number of partitions for splitting
    lambdas: list of lambdas to test on
    
    """
    

    #calculate means, variance,  standardize X
    def reduce_mean(row1, row2):
        s1 = row1[0]
        s2 = row2[0]
        
        return (s1 + s2, s1 / (s1 + s2) * row1[1] + s2 / (s1 + s2) * row2[1])
    
    #vector of means of length p    
    means_X = np.array(Data.map(lambda row: (1, row[0])).reduce(reduce_mean).collect()) 
    mean_Y = Data.map(lambda row: (1, row[1])).reduce(reduce_mean)
    
    p = len(means_X)
    
    #center X
    Data.map(lambda row: (np.arrray([row[0][i] - means_X[i] for i in range(p)]), row[1]))
    
    # vector of variance of X of length p (using the fact that now X is centered)
    vars_X = np.array(Data.map(lambda row: (1, row[0]**2)).reduce(reduce_mean).collect())
    
    #standardize X
    Data.map(lambda row: (np.array([row[1][i] / vars_X[i] for i in range(p)]), row[1]))
    
        
    def map_statistics(row):
        # calculate statistics for one row [size, mean(x), mean(y), Y^TY, y * x, cov(x)]
        return (np.random.randint(k), [1, x, y, y**2, y * x, np.zeros((len(row), len(row)))])

    statistics = Data.map(map_statistics)

    def reduce_statistics(row1, row2):
        s_1 = row1[0]
        s_2 = row2[0]
        mean_x = s_1 / (s_1 + s_2) * row1[1] + s_2 / (s_1 + s_2) * row2[1]
        mean_y = s_1 / (s_1 + s_2) * row1[2] + s_2 / (s_1 + s_2) * row2[2]
        cov = s_1 / (s_1 + s_2) * row1[5] + s_2 / (s_1 + s_2) * row2[5] + s_1 * s_2 / (
            s_1 + s_2)**2 * (row1[1] - row2[1]).T.dot(row1[1] - row2[1])
        emit = [s_1 + s_2, mean_x, mean_y, row1[3] +
                row2[3], row1[4] + row2[4], cov]
        return emit

    statistics = statistics.reduceByKey(reduce_statistics)

    # Cross validation
    if False:
        for i in range(k):
            statistics_train = statistics.filter(lambda row: row[0] != i)
            statistics_train = statistics_train.reduce(reduce_statistics)
            statistics_test = statistics.filter(lambda row: row[0] == i)

    return statistics

SyntaxError: invalid syntax (<ipython-input-2-d400f3a938b1>, line 25)

In [9]:
test_data = sc.parallelize([(np.array([i]), i) for i in range(1, 10)])
test_data.take(4)

[(array([1]), 1), (array([2]), 2), (array([3]), 3), (array([4]), 4)]

In [10]:
stat = PenalizedLR_MR(test_data, 1, [1])
stat.collect()

[(0, [9, array([[5.]]), 5.0, 285, array([[285]]), array([[6.66666667]])])]

In [11]:
np.cov(np.arange(1,10), bias=True)

array(6.66666667)